### 문제 1-2 : 2단계 Multi Chain 만들기 - 여행지 정보 시스템

#### 1) 각 단계의 결과를 모두 출력

In [ ]:
from dotenv import load_dotenv
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

from pprint import pprint

# .env 파일 불러와서 환경변수로 설정
load_dotenv(dotenv_path='.env')
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Step 1: 사용자가 입력한 지역에 따라 관광 명소 추천
prompt1 = ChatPromptTemplate.from_messages(
    [ ("system", "당신은 관광 정보를 제공하는 여행사 직원입니다.") ,
     ("user", "{area} 지역의 대표적인 관광 명소를 1가지 추천해 주세요. 명소 이름만 남겨주세요.") ]
)

# Step 2: 추천된 명소의 상세정보 제공
prompt2 = ChatPromptTemplate.from_messages(
    [ ("system", "당신은 관광 정보를 제공하는 여행사 직원입니다.") ,
     ("user", "{spot} 추천한 1개 명소의 상세정보(역사, 특징, 방문 팁 등)를 알려주세요. 추천한 명소의 이름을 밝히면서 문장을 시작해 주세요.") ]
)

# OpenAI 모델 사용
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7
)

# 체인 1: 명소 추천
chain1 = prompt1 | llm | StrOutputParser()

# 체인 1 결과 확인
area = chain1.invoke({"area": "일본"})
print("===> 1단계 결과:")
print(area)


===> 1단계 결과:
후지산


In [ ]:
# 체인 2: 명소 상세정보 제공
chain2 = (
    {"spot": chain1}  # chain1의 출력을 spot 변수로 전달
    | prompt2
    | llm
    | StrOutputParser()
)

# 체인 2 결과 확인
spot_detail = chain2.invoke({"area": "일본"})
print("===> 2단계 결과:")
pprint(spot_detail)

===> 2단계 결과:
('후시미 이나리 신사(伏見稲荷大社)는 교토 남쪽, 후시미(伏見) 지역에 자리한 인ari(稲荷) 신을 모신 일본 대표 신사입니다.  \n'
 '\n'
 '▶ 역사  \n'
 '- 창건은 711년으로, 헤이안 시대(794년) 교토가 수도가 되며 천확의 신앙을 받기 시작했습니다.  \n'
 '- ‘인사이(稲荷)’라는 이름은 ‘곡식이 무르익는다(稲が成る)’에서 유래해 풍작·상업 번창의 신으로 숭배돼 왔습니다.  \n'
 '- 1000년 이상 지속된 신앙은 민간에 깊게 뿌리내려 현재 일본 전국에 약 3만 개 소(分社)가 있으며, 본사는 바로 후시미 이나리 '
 '신사입니다.  \n'
 '\n'
 '▶ 특징  \n'
 '1. 천 개의 적색 도리이(鳥居)  \n'
 '   - 산 전체를 둘러싼 4 km의 산책로에 무려 약 1만 기의 도리이가 늘어서 있습니다.  \n'
 '   - 도리이는 개인·기업이 소원을 이루면 감사의 헌납물로 세워지며, 뒷면에 기부자 이름이 쓰여 있습니다.  \n'
 '2. 등산·하이킹이 가능한 ‘이나리산(233 m)’  \n'
 '   - 정상까지 왕복 2~3시간. 도중 ‘요쓰츠녈미샤(四ツ辻宮)’에서 교토 시가지·후지산(날씨 좋은 날) 조망 가능.  \n'
 '3. 여우(키츠네) 조각  \n'
 '   - 여우는 신의 사자(使者)로 여겨져 신사 곳곳에 돌·나무 여우상이 있습니다. 입에 낀 쌀알·열쇠는 풍작·재산 상징.  \n'
 '4. 야쓰카(八重塔)·국보 건물  \n'
 '   - 본전·괴전 등 5개 건물이 중요문화재, 2017년엔 랜드마크 타워 ‘료칸타워’도 완공돼 밤경관도 인기.  \n'
 '\n'
 '▶ 방문 팁  \n'
 '- 시간: 평일 오전 7시~오후 6시 30분(입장료 무료).  \n'
 '- 미관: 도리이 터널 사진은 역광 피해 오전 8시 전·오후 4시 이후가 가장 좋습니다.  \n'
 '- 피크 회피: 단체관광객이 몰리는 10~11시는 피하고, 오전 6~8시 또는 저녁 5시 이후 방문하면 한적함을 만끽.  

###### 이처럼 각 단계의 결과를 모두 출력하면 체인이 일관적으로 연결되지 않는다.

#### 2) Chain2의 결과만 출력

In [30]:
from dotenv import load_dotenv
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

from pprint import pprint

# .env 파일 불러와서 환경변수로 설정
load_dotenv(dotenv_path='.env')
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Step 1: 사용자가 입력한 지역에 따라 관광 명소 추천
prompt1 = ChatPromptTemplate.from_messages(
    [ ("system", "당신은 관광 정보를 제공하는 여행사 직원입니다.") ,
     ("user", "{area} 지역의 대표적인 관광 명소를 1가지 추천해 주세요. 명소 이름만 남겨주세요.") ]
)

# Step 2: 추천된 명소의 상세정보 제공
prompt2 = ChatPromptTemplate.from_messages(
    [ ("system", "당신은 관광 정보를 제공하는 여행사 직원입니다.") ,
     ("user", "{spot} 추천한 1개 명소의 상세정보(역사, 특징, 방문 팁 등)를 알려주세요. 추천한 명소의 이름을 밝히면서 문장을 시작해 주세요.") ]
)

# OpenAI 모델 사용
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7
)

# 체인 1: 명소 추천
chain1 = prompt1 | llm | StrOutputParser()

## 체인 2: 명소 상세정보 제공
chain2 = (
    {"spot": chain1}  # chain1의 출력을 spot 변수로 전달
    | prompt2
    | llm
    | StrOutputParser()
)

# 체인 2 결과 확인
spot_detail = chain2.invoke({"area": "일본"})
print("===> 2단계 결과:")
pprint(spot_detail)


===> 2단계 결과:
('후시미 이나리 신사(伏見稲荷大社)는 711년 창건된 일본 대표 신사로, ‘이나리(稲荷)’ 즉 ‘곡물·풍요의 신’을 모신 4만여 개 이나리 '
 '신사의 총본산이며, 1300년 가까운 세월 동안 민간 신앙의 중심지로 사랑받아 온 역사적 명소입니다.\n'
 '\n'
 '1. 역사\n'
 '- 창건 당시는 하라(秦) 씨 집안이 화려한 베 문화(직조·무역)로 번영했던 시기로, 하라 씨의 수호신이었던 이나리 신이 전국으로 '
 '확산되며 ‘기업 번창·가내 안전’의 상징이 되었습니다.\n'
 '- 1499년 본전(本殿)과 1589년 도요토미 히데요시가 현존하는 주요 전각(本殿·楼門·中門·摂社)을 대규모로 증축, 화려한 ‘도요토미 '
 '진즉(桃山) 양식’을 완성해 오늘날까지 보존되고 있습니다.\n'
 '- 약 1만 개의 ‘토리이(鳥居)’가 험준 산 허리를 따라 4km에 걸쳐 늘어선 ‘천본立(千本鳥居)’는 에도~메이지 시대에 걸쳐 '
 '기업·가정의 소원 성취 감사 제작 방식으로 늘어난 것이 특징입니다.\n'
 '\n'
 '2. 특징\n'
 '- ‘토리이 터널’은 후시미 이나리의 상징으로, 검은 기둥 위에 형형색색의 주황색 도료 ‘베르밀리온’을 칠해 어둠 속에서도 길을 '
 '인도한다는 의미를 담고 있으며, 사진 찰영 포인트로도 세계적으로 유명합니다.\n'
 '- ‘오모쿠데(御守狐)’라 불리는 여우 동상이 곳곳에 있어, 입에 쌀알갱이·비단· 열쇠 등을 물고 있는 모습으로 ‘기원 성취’의 매개체 '
 '역할을 합니다.\n'
 '- 산 전체가 ‘이나리 성산(稲荷聖山)’이라는 명으로 신앙 대상이므로, 정상(233m)에 오르면 교토 시가(祇園·가와라마치)와 산요·산인 '
 '쪽 연락 고속도로까지 파노라마 조망이 가능합니다.\n'
 '\n'
 '3. 방문 팁\n'
 '- 시간: 24시간 산책로가 개방되지만, 사무국(부적·수양)은 8:30~16:30, 야간 조명은 없으므로 해질녘까지만 터널 산책을 '
 '추천합니다.\n'
 '- 소요 시간: 입구~천본鳥居·